In [1]:
import pickle
import random
import operator
import numpy as np
import pandas as pd
import seaborn as sns
from collections import deque
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
class Question:
    def __init__(self, col, value):
        self.col = col
        self.value = value
        
    def match(self, row):
        val = row[self.col]
        if isNumeric(val):    
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        if isNumeric(self.value):
            condition = '>='
        else:
            condition = '=='
        return "Is %s %s %s?" % (header[self.col], condition, str(self.value))

In [3]:
class Queue:
    def __init__(self):
        self.buffer = deque()
    
    def enqueue(self, val):
        self.buffer.appendleft(val)
        
    def dequeue(self):
        return self.buffer.pop()
    
    def is_empty(self):
        return len(self.buffer)==0
    
    def size(self):
        return len(self.buffer)

In [4]:
def getUnique(rows, col):
    return set([row[col] for row in rows])
    
    
    
def labelCounts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 1
        else:
            counts[label] += 1
            
    return counts



def isNumeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [5]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
            
    return true_rows, false_rows



def gini(rows):
    counts = labelCounts(rows) #counts is dictionary
    
    impurity = 1
    for label in counts:
        probability = counts[label]/float(len(rows)) 
        impurity -= (probability * probability)       # 1 - np.sum(p**2)
        
    return impurity



def informationGain(left, right, currentUncertinity):
    totalLength = len(left) + len(right)
    probability_left = len(left)/float(totalLength)
    probability_right = 1 - probability_left
    weightedGini = (probability_left * gini(left)) + (probability_right * gini(right))
    info_Gain = currentUncertinity - weightedGini
    return info_Gain

In [6]:
def Buble_sorting(all_QuestionGain):
    new_Question_Gain_Order = []
    Q_list = []
    G_list = []
    for Question, Gain in all_QuestionGain:
        Q_list.append(Question) 
        G_list.append(Gain)
    
    tempGain = G_list.copy()
    tempQues = Q_list.copy()
    
    l = len(all_QuestionGain)
    new_Gain_Order = l* [0]
    new_Ques_Order = l* [0]


    for i in range(l):
        new_Gain_Order[i] = tempGain[0]
        for j in range(l-i-1):
            if tempGain[j+1] > new_Gain_Order[i]:
                new_Gain_Order[i] = tempGain[j+1]
        
        Ques_index = tempGain.index(new_Gain_Order[i])
        new_Ques_Order[i] = tempQues[Ques_index]
        
        new_Question_Gain_Order.append( (new_Ques_Order[i],new_Gain_Order[i]) )
        
        tempGain.remove(new_Gain_Order[i])
        tempQues.remove(new_Ques_Order[i])

    return new_Question_Gain_Order

In [7]:
def findBestSplit(rows):
    Question_Gain = []
    
    currentUncertinity = gini(rows)
    no_feature = len(rows[0]) - 1    #if a row has 5 columns and the last column is label, then total feature no = 5-1
    
    for col in range(no_feature):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            trueRows, falseRows = partition(rows, question)

            if ( (len(trueRows) == 0) or (len(falseRows) == 0) ):
                continue

            gain = informationGain(trueRows, falseRows, currentUncertinity)
            Question_Gain.append((question,gain))
            
            
    if(len(Question_Gain) >= 1):
        Question_Gain = Buble_sorting(Question_Gain)
        bestQuestion = Question_Gain[0][0]
        bestGain = Question_Gain[0][1]
    else:
        bestQuestion = None
        bestGain = 0
        
    
    return bestGain, bestQuestion, Question_Gain

In [8]:
def Build_Node_Branches(rows):
    bestGain, bestQuestion, Question_Gain = findBestSplit(rows)
    
    if (bestGain == 0):
        leaf = labelCounts(rows)
        return leaf,[],[]
    else:
        trueRows, falseRows = partition(rows, bestQuestion)
        return bestQuestion, trueRows, falseRows
    
    

def Build_Q_Tree(data):
    TF_Branches = Queue()
    Ques_q = Queue()
    
    TF_Branches.enqueue( (data,'0') )
    while(TF_Branches.is_empty() == False):
        rows, Target_Level = TF_Branches.dequeue()
        Q,T,F = Build_Node_Branches(rows)

        if ( (len(T) !=0) and (len(F) !=0) ):
            TF_Branches.enqueue( (T, Target_Level+'1') )
            TF_Branches.enqueue( (F, Target_Level+'0') )

        Ques_q.enqueue( (Q, Target_Level) )
    
    return Ques_q

In [9]:
def Print_Tree(Ques_q):
    q_list = list(Ques_q.buffer)
    q_list.reverse()
    Tree_Display(q_list.copy(), '', )
    
    

def Find_Target_Ques(a_list, Target_Level):
    for ques,level in a_list:
        if (level==Target_Level):
            return ques
        
        

def Tree_Display(a_list, spacing, Target_Level='0'):
    ques = Find_Target_Ques(a_list, Target_Level)
    
    if(isinstance(ques, dict)):
        print(spacing + '|--Predict: ', ques)
        a_list.remove((ques, Target_Level))
        return
    
    spacing += '|----'
    print(spacing, ques)
    print(spacing + '|>True')
    a_list.remove((ques, Target_Level))
    Tree_Display(a_list, spacing, Target_Level+'1')
    
    print(spacing + '|>False')
    Tree_Display(a_list, spacing, Target_Level+'0')
    
    

def Print_Leaf(counts):
    probability = {}
    total_N_Outputs = sum(counts.values())
    
    for lbl in counts:
        probability[lbl] = str( (counts[lbl] / total_N_Outputs)*100 ) + '%'
    
    return probability   



def Predict(row, Ques_q):
    q_list = list(Ques_q.buffer)
    q_list.reverse()
    
    copy_que = Queue()
    for x in q_list:
        copy_que.enqueue(x)
    
    Target_Level = '0'
    while(Ques_q.is_empty() == False):
        ques, level = copy_que.dequeue()
        if (level == Target_Level):
            if(isinstance(ques, dict)):
                return ques
            
            else:
                if ques.match(row):
                    Target_Level = level + '1'
                else:
                    Target_Level = level + '0'

In [10]:
def Performance_Report(y_test, y_pred, label_names=[], index=[], columns=[]):
    cMatrix = confusion_matrix(y_test, y_pred)
    
    if ( (len(index) > 0) and (len(columns) > 0) ):
        cm_df = pd.DataFrame(cMatrix, index, columns)

        plt.figure(figsize=(5,4))
        sns.heatmap(cm_df, annot=True)
        plt.title('Confusion Matrix')
        plt.ylabel('Actal Values')
        plt.xlabel('Predicted Values')
        plt.show()
    
    if ( len(label_names) > 0 ):
        print(classification_report(y_test, y_pred, target_names=label_names))
        
    return accuracy_score(y_test, y_pred)

In [11]:
class Decesion_QTree:
    def __init__(self):
        pass
    
    def add_XY(self):
        self.data = []
        for i in range(len(self.X)):
            row = []
            row.extend(self.X[i])
            row.append(str(self.Y[i]))
            self.data.append(row)
        
        
    def Build(self, X,Y):
        self.X = X
        self.Y = Y
        self.add_XY()
        self.Q_tree = Build_Q_Tree(self.data)
        
        
    def Print(self):
        Print_Tree(self.Q_tree)
        
        
    def Leaf(self, features, percentage = ''):
        self.features = features
        predict = Predict(self.features, self.Q_tree)
        
        if(percentage == '%'):
            print(Print_Leaf(predict))
        else:
            print(predict)
            
            
    def Predict(self, features = ''):
        if(len(features) == 0):
            features = self.features
            
        predict = Predict(features, self.Q_tree)
        sorted_predict = sorted(predict.items(), key=operator.itemgetter(1))
        final_predict = sorted_predict[-1][0]
        
        if (final_predict.isnumeric()):
            final_predict = int(final_predict)
            
        return final_predict

In [12]:
path = 'D:/Work/Programming Languages/Python/JupyterLab/Decision Tree/Datasets/'
pickleIN = open(path + 'BrestCancer_Train_X.pickle', 'rb') 
X = pickle.load(pickleIN)
pickleIN = open(path + 'BrestCancer_Train_Y.pickle', 'rb') 
Y = pickle.load(pickleIN)

header = list(X.columns.values)
header.append('Label')

X = X.values.tolist()
Y = list(Y)

X_train, x_test, Y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=50)


label_names = ['Have']


print(len(X_train))
print(len(x_test))
print(np.array(X_train).shape)

340
86
(340, 30)


In [13]:
myTree = Decesion_QTree()
myTree.Build(X_train, Y_train)
myTree.Print()

|---- Is concave points_mean >= 0.2575546719681908?
|----|>True
|----|---- Is perimeter_worst >= 0.26390756511778474?
|----|----|>True
|----|----|---- Is concavity_worst >= 0.17651757188498404?
|----|----|----|>True
|----|----|----|---- Is fractal_dimension_se >= 0.4175200033166121?
|----|----|----|----|>True
|----|----|----|----|--Predict:  {'0': 1}
|----|----|----|----|>False
|----|----|----|----|---- Is texture_mean >= 0.1565776124450456?
|----|----|----|----|----|>True
|----|----|----|----|----|--Predict:  {'1': 111}
|----|----|----|----|----|>False
|----|----|----|----|----|---- Is radius_mean >= 0.4012967958729709?
|----|----|----|----|----|----|>True
|----|----|----|----|----|----|--Predict:  {'1': 1}
|----|----|----|----|----|----|>False
|----|----|----|----|----|----|--Predict:  {'0': 1}
|----|----|----|>False
|----|----|----|--Predict:  {'0': 2}
|----|----|>False
|----|----|---- Is texture_worst >= 0.3827292110874201?
|----|----|----|>True
|----|----|----|--Predict:  {'1': 4}

In [14]:
myTree.Leaf(x_test[0])

{'1': 111}


In [15]:
myTree.Leaf(x_test[0], "%")

{'1': '100.0%'}


In [16]:
y_pred_QTree = []
for x in x_test:
    y_pred_QTree.append(myTree.Predict(x))
    
    
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
y_pred_library = list(decision_tree.predict(x_test))

In [17]:
Performance_Report(y_test, y_pred_QTree)

0.8837209302325582

In [18]:
Performance_Report(y_test, y_pred_library)

0.8953488372093024